# Sales Manager conversions

In this task we need to calculate conversion rate from touch to deal and to find the best manager.

In [1]:
import pandas as pd

df = pd.read_csv("best_salesman_homework.csv")

df.head()

,client_account_id,date,event_name,manager_id,manager_nickname
0,0,2022-05-09,first_touch,1.0,Justin Beiber
1,1,2022-03-21,first_touch,3.0,Joe Biden
2,2,2022-04-18,first_touch,2.0,Kylie Jenner
3,3,2022-02-07,first_touch,2.0,Kylie Jenner
4,4,2022-04-08,first_touch,1.0,Justin Beiber


In [2]:
df.shape

(3183, 5)

In [32]:
df['date'] = pd.to_datetime(df['date'])

We can see that most of the clients remain in the status 'first_touch'.
We can make two tables:
- the first table with only 'first_touch' cliets;
- the second table with clients achieved 'deal' status.

In [53]:
# customers with only 'first_touch' status
v = df.client_account_id.value_counts()
df1 = df[df.client_account_id.isin(v.index[v.eq(1)])]
df1.shape[0]

2791

In [56]:
# customers with 'deal' status
v = df.client_account_id.value_counts()
df2 = df[df.client_account_id.isin(v.index[v.gt(1)])]
df2.head()

,client_account_id,date,event_name,manager_id,manager_nickname
40,40,2022-05-02,first_touch,1.0,Justin Beiber
41,40,2022-02-24,deal,3.0,Joe Biden
64,63,2022-02-03,first_touch,1.0,Justin Beiber
65,63,2022-03-14,deal,2.0,Kylie Jenner
88,86,2021-12-17,first_touch,3.0,Joe Biden


Let's modify the 2nd dataframe and make one row for one customer.

In [79]:
df_first_touch = df2[df2['event_name'] == 'first_touch']
df_deal = df2[df2['event_name'] == 'deal']
df_merged = df_first_touch.merge(df_deal, on='client_account_id', how = 'left')

I noticed that for some customers the 'deal' status happened earlier than 'one_touch'. 
Probably there is a mistake. If the customer already made a purchase, there was obviously some touchpoint before the 'deal', that is why the 'first_touch' for this customer looses its value. 
As in this task we calculate conversion rate for the first touch customers, we can delete customers who do not meet these conditions.

In [80]:
df_merged['date_diff'] = (df_merged['date_y'] - df_merged['date_x']).astype('int64')//86400000000000

In [81]:
df_merged.head()

,client_account_id,date_x,event_name_x,manager_id_x,manager_nickname_x,date_y,event_name_y,manager_id_y,manager_nickname_y,date_diff
0,40,2022-05-02,first_touch,1.0,Justin Beiber,2022-02-24,deal,3.0,Joe Biden,-67
1,63,2022-02-03,first_touch,1.0,Justin Beiber,2022-03-14,deal,2.0,Kylie Jenner,39
2,86,2021-12-17,first_touch,3.0,Joe Biden,2022-05-24,deal,1.0,Justin Beiber,158
3,87,2022-05-30,first_touch,1.0,Justin Beiber,2022-05-23,deal,2.0,Kylie Jenner,-7
4,127,2022-05-22,first_touch,1.0,Justin Beiber,2021-08-30,deal,3.0,Joe Biden,-265


In [74]:
index_names = df_merged[df_merged['date_diff'] < 0].index
df_updated = df_merged.drop(index_names)
df_updated.shape[0]

116

In [83]:
accounts_to_del = list(df_merged[df_merged['date_diff'] < 0]['client_account_id'])

Now we can calculate **conversion rate**.

$$
  CR = \frac{Number of deals}{Number of touches} 100
$$

In [75]:
CR = df_updated.shape[0]/(df_updated.shape[0] + df1.shape[0]) * 100
print(f'Conversion Rate for the whole Sales group: {CR}')

Conversion Rate for the whole Sales group: 3.990368077055383


Let's calculate conversion rate for each manager.

In [67]:
df.manager_id.unique()

array([1., 3., 2.])

In [86]:
df_manager_1 = df[df['manager_id'] == 1.0]
df_manager_1.head()

,client_account_id,date,event_name,manager_id,manager_nickname
0,0,2022-05-09,first_touch,1.0,Justin Beiber
4,4,2022-04-08,first_touch,1.0,Justin Beiber
20,20,2022-04-20,first_touch,1.0,Justin Beiber
25,25,2022-05-15,first_touch,1.0,Justin Beiber
26,26,2022-02-24,first_touch,1.0,Justin Beiber


In [87]:
df_manager_1 = df_manager_1[~df_manager_1.client_account_id.isin(accounts_to_del)]

As we need to calculate conversion rate from touch to deal, we will exclude the clients with the deal status if first_touch was performed by another manager.

In [94]:
v1 = df_manager_1.client_account_id.value_counts()
df_manager_1_deal = df_manager_1[df_manager_1.client_account_id.isin(v1.index[v1.gt(1)])]
df_manager_1_touch = df_manager_1[df_manager_1.client_account_id.isin(v1.index[v1.eq(1)])]

In [96]:
CR1 = (df_manager_1_deal.shape[0]/2) / (df_manager_1_deal.shape[0]/2 + df_manager_1_touch.shape[0]) * 100
print(f'Conversion Rate for Justin Beiber: {CR1}')

Conversion Rate for Justin Beiber: 0.9101251422070534


In [97]:
df_manager_2 = df[df['manager_id'] == 2.0]
df_manager_2.head()

,client_account_id,date,event_name,manager_id,manager_nickname
2,2,2022-04-18,first_touch,2.0,Kylie Jenner
3,3,2022-02-07,first_touch,2.0,Kylie Jenner
5,5,2021-09-29,first_touch,2.0,Kylie Jenner
6,6,2022-04-13,first_touch,2.0,Kylie Jenner
8,8,2022-02-23,first_touch,2.0,Kylie Jenner


In [101]:
df_manager_2 = df_manager_2[~df_manager_2.client_account_id.isin(accounts_to_del)]

In [102]:
v2 = df_manager_2.client_account_id.value_counts()
df_manager_2_deal = df_manager_2[df_manager_2.client_account_id.isin(v2.index[v2.gt(1)])]
df_manager_2_touch = df_manager_2[df_manager_2.client_account_id.isin(v2.index[v2.eq(1)])]

In [103]:
CR2 = (df_manager_2_deal.shape[0]/2) / (df_manager_2_deal.shape[0]/2 + df_manager_2_touch.shape[0]) * 100
print(f'Conversion Rate for Kylie Jenner: {CR2}')

Conversion Rate for Kylie Jenner: 1.2752391073326248


In [107]:
df_manager_3 = df[df['manager_id'] == 3.0]
df_manager_3.head()

,client_account_id,date,event_name,manager_id,manager_nickname
1,1,2022-03-21,first_touch,3.0,Joe Biden
7,7,2021-07-31,first_touch,3.0,Joe Biden
9,9,2021-12-15,first_touch,3.0,Joe Biden
11,11,2022-05-21,first_touch,3.0,Joe Biden
12,12,2022-05-29,first_touch,3.0,Joe Biden


In [108]:
df_manager_3 = df_manager_3[~df_manager_3.client_account_id.isin(accounts_to_del)]

In [109]:
v3 = df_manager_3.client_account_id.value_counts()
df_manager_3_deal = df_manager_3[df_manager_3.client_account_id.isin(v3.index[v3.gt(1)])]
df_manager_3_touch = df_manager_3[df_manager_3.client_account_id.isin(v3.index[v3.eq(1)])]

In [110]:
CR3 = (df_manager_3_deal.shape[0]/2) / (df_manager_3_deal.shape[0]/2 + df_manager_3_touch.shape[0]) * 100
print(f'Conversion Rate for Joe Biden: {round(CR3)}%')

Conversion Rate for Joe Biden: 1.9827586206896552


**Our winner is Joe Biden**

However, we see that the conversion rate for the whole Sales group is higher than for individual Sales Managers.
It happend because we excluded the cases when one customer was supervised by different managers.

As our team does so well as a group, maybe we need to reconsider our KPIs and to count such cases for the benefit of the managers too.